In [2]:
# libraries imports
import pandas as pd
import numpy as np
import datetime as dt
import openpyxl
import html5lib
print(pd.__version__)
print(openpyxl.__version__)


1.3.5
3.0.7


## Testing datetime in pandas

In [23]:
d = {'date': ['1/2/2000'], 'date2': ['01/02/2000'], 'date3': ['1-2-2000'], 'date4': ['1.2.2000']}
df = pd.DataFrame(data=d)
print(df.dtypes)
df = df.apply(pd.to_datetime)
print(df.dtypes)
df

date     object
date2    object
date3    object
date4    object
dtype: object
date     datetime64[ns]
date2    datetime64[ns]
date3    datetime64[ns]
date4    datetime64[ns]
dtype: object


,date,date2,date3,date4
0,2000-01-02,2000-01-02,2000-01-02,2000-01-02


## Testing datetime in pandas - with UK dates one per series

In [25]:
d = {'date': ['5/2/2000'], 'date2': ['3/02/2000'], 'date3': ['1-2-2000'], 'date4': ['10.2.2000']}
df = pd.DataFrame(data=d)
print(df.dtypes)

# df[['date', 'date2', 'date3', 'date4']] = df[['date', 'date2', 'date3', 'date4']].apply(pd.to_datetime)   #  WORKS but not UK dates
df[["date", "date2", "date3", "date4"]] = df[["date", "date2", "date3", "date4"]].apply(pd.to_datetime, dayfirst=True)   #  WORKS
# df[['date', 'date2', 'date3', 'date4']] = df[['date', 'date2', 'date3', 'date4']].apply(lambda x: pd.to_datetime(x, dayfirst=True))   # WORKS
print(df.dtypes)
df
# No problem


date     object
date2    object
date3    object
date4    object
dtype: object
date     datetime64[ns]
date2    datetime64[ns]
date3    datetime64[ns]
date4    datetime64[ns]
dtype: object


,date,date2,date3,date4
0,2000-02-05,2000-02-03,2000-02-01,2000-02-10


## Now with all UK in one series

In [5]:
d = ['13/2/2000', '14/02/2000', '15/2/2000', '16/2/2000']
s = pd.Series(d)
s = pd.to_datetime(s)
s

# Again pandas works out these are UK dates because they are all clearly in the same style with first digit above 12.

0   2000-02-13
1   2000-02-14
2   2000-02-15
3   2000-02-16
dtype: datetime64[ns]

## Now UK dates which are less clear whether they are UK or US style.

In [6]:
d = ['1/2/2000', '2/02/2000', '2-3-2000', '16.2.2000']
s = pd.Series(d)
s = pd.to_datetime(s)
s

# Three of the dates could be US or UK dates and these give pandas a problem. But 16/2/2000 was spotted as a UK date.

0   2000-01-02
1   2000-02-02
2   2000-02-03
3   2000-02-16
dtype: datetime64[ns]

In [7]:
d = ['1/2/2000', '2/02/2000', '2-3-2000', '16.2.2000']
s = pd.Series(d)
s = pd.to_datetime(s, dayfirst=True)
s

# The fix to this problem is the dayfirst parameter.

0   2000-02-01
1   2000-02-02
2   2000-03-02
3   2000-02-16
dtype: datetime64[ns]

## Testing using a range of date styles

Lets push Pandas on the range of formats that is can import dates without having to use the format parameter.

In [8]:
http = r'https://www.ibm.com/docs/en/cmofz/10.1.0?topic=SSQHWE_10.1.0/com.ibm.ondemand.mp.doc/arsa0257.htm'
table = pd.read_html(http)
df = table[0]
df

# test which datestyles pandas can convert
df['Example_clean'] = pd.to_datetime(df['Example'])
print(df.dtypes)
df
# yes it converted all of them!!

Format Specifier            object
Format                      object
Example                     object
Example_clean       datetime64[ns]
dtype: object


,Format Specifier,Format,Example,Example_clean
0,%m/%d/%Y,mm/dd/yyyy,02/21/2018,2018-02-21 00:00:00
1,%m/%d/%y,mm/dd/yy,02/21/18,2018-02-21 00:00:00
2,%d/%m/%Y,dd/mm/yyyy,21/02/2018,2018-02-21 00:00:00
3,%d/%m/%y,dd/mm/yy,21/02/18,2018-02-21 00:00:00
4,%d-%m-%Y,dd-mm-yyyy,21-02-2018,2018-02-21 00:00:00
5,%d-%m-%y,dd-mm-yy,21-02-18,2018-02-21 00:00:00
6,%m-%d-%Y,mm-dd-yyyy,02-21-2018,2018-02-21 00:00:00
7,%m-%d-%y,mm-dd-yy,02-21-18,2018-02-21 00:00:00
8,%Y-%m-%d,yyyy-mm-dd,2018-02-21,2018-02-21 00:00:00
9,%f/%e/%Y,m/d/yyyy,2/21/2018,2018-02-21 00:00:00


## Now with some junk that can not be converted to dates

In [18]:
d = ['1/2/2000', '2/02/2000', None, 'test']
s = pd.Series(d)
s = pd.to_datetime(s)
s

# ParserError: Unknown string format: test
# Pandas does not recognise 'test' as a date.


0   2000-01-02
1   2000-02-02
2          NaT
dtype: datetime64[ns]

In [10]:
# But using the errors='coerce' parameter fixes this by filling the none dates with NaT (not a time)

d = ['1/2/2000', '2/02/2000', None, 'test']
s = pd.Series(d)
s = pd.to_datetime(s, errors='coerce')
s

0   2000-01-02
1   2000-02-02
2          NaT
3          NaT
dtype: datetime64[ns]

## Test using very messy dates

In [11]:
# taken a sample from https://stackoverflow.com/questions/6194285/dealing-with-messy-dates

messy_dates = ["May2/ 12 noon", "9:45 am", "11:00 AM AST", "April 27 / 12:00 AST", 
"11:40 AM AST", "April 25 2011", "April 12th 2011 / 8:44", "April 12 2011 / 8:36am", 
"April 12 2011 / 8:30am", "April 12th 2011 / 8:18", "April 12 2011 / 8:12am", 
"April 11th 2011 / 5:57pm", "April 11th 2011 / 5:49pm", "April 11th 2011 / 5:42pm", 
"April 11th 2011 / 5:36pm", "April 11th 2011 / 5:27", "April 5 @ 11:26am", 
"8:50", "April 4th 12:45pm", "April 4th around 10am", "April 4th around 10am", 
"Mar 18, 2011 9:33am", "Mar 18, 2011 9:27am", "df", "fg", "12:16", 
"9:50", "Feb 8, 2011 / 12:20pm", "8:34 am  2/4/11", "Jan 31, 2011 2:50pm", 
"Jan 31, 2011 2:45pm", "Jan 31, 2011 2:38pm", "Jan 31, 2011 2:26pm", 
"11h09", "11:00 am", "1h02 pm", "10h03", "2h10", "Jan 13, 2011 9:50am Van", 
"Jan 12, 2011", "Jan 12, 2011 3:59pm", "Jan 12     14:19PM", 
"Jan 12, 2011 1:35pm", "Jan 12,2011 1:28pm", "1h36", "9h15", 
"9h09", "8h51", "8h45", "8h35", "1h12 pm", "12h59", "11h52 am", 
"10h45", "15h55", "Dec 31, 10 11:11am", "Dec 31,10 10:15am"]

In [12]:
s = pd.Series(messy_dates)
s = pd.to_datetime(s, errors='coerce')
# len(s)            # 57 total
# s.count()         # 30 dates
# s.isna().sum()    # 27 NaT


C:\Users\goldsby_c\AppData\Local\Programs\Python\Python39\lib\site-packages\dateutil\parser\_parser.py:1213: UnknownTimezoneWarning: tzname AST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [13]:
# So which ones worked and did not?

df = pd.DataFrame({'dates': messy_dates})
# print(df.dtypes)
df['clean_dates'] = pd.to_datetime(df['dates'], errors='coerce')

# ones that where not converted 
df.loc[df['clean_dates'].isna()]

# ones that were converted - Note times on their own were given todays date
df.loc[df['clean_dates'].notna()]

# So 22 of the 57 are correct.

,dates,clean_dates
1,9:45 am,2022-04-18 09:45:00
2,11:00 AM AST,2022-04-18 11:00:00
4,11:40 AM AST,2022-04-18 11:40:00
5,April 25 2011,2011-04-25 00:00:00
6,April 12th 2011 / 8:44,2011-04-12 08:44:00
7,April 12 2011 / 8:36am,2011-04-12 08:36:00
8,April 12 2011 / 8:30am,2011-04-12 08:30:00
9,April 12th 2011 / 8:18,2011-04-12 08:18:00
10,April 12 2011 / 8:12am,2011-04-12 08:12:00
11,April 11th 2011 / 5:57pm,2011-04-11 17:57:00


# Import csv file and parse dates

In [14]:

data = r'../GitHub/dates.csv'

# by default dates are seen as strings and Pandas gives object type
df = pd.read_csv(data)
print(df.dtypes)

# parse_dates parameter is the magic for getting pandas to auto process dates. It can be handed a list of column names 
df = pd.read_csv(data, parse_dates=['date1', 'date2']) 
# or column indexes.
# df = pd.read_csv(data, parse_dates=[0, 1])  

# using the infer_datetime_format can speed up importing but can only be used if dates are consistent
df = pd.read_csv(data, parse_dates=[0, 1], infer_datetime_format=True)  
print(df.dtypes)
df.head()

date1    object
date2    object
dtype: object
date1    datetime64[ns]
date2    datetime64[ns]
dtype: object


,date1,date2
0,2021-11-02,2021-11-02
1,2021-11-10,2021-11-10
2,2021-11-15,2021-11-15
3,2021-11-17,2021-11-17
4,2021-11-18,2021-11-18


# Import excel and parse dates

Luckily read_excel and read_csv are consistent in using the parse_dates parameter.

In [15]:
data = r'../GitHub/dates.xlsx'

# by default dates are seen as strings and Pandas gives object type
df = pd.read_excel(data)
print(df.dtypes)

# parse_dates parameter is the magic for getting pandas to auto process dates. It can be handed a list of names or col numbers
df = pd.read_excel(data, parse_dates=['date1', 'date2']) 
# or
df = pd.read_excel(data, parse_dates=[0, 1])  
print(df.dtypes)
df.head()

date1    object
date2    object
dtype: object
date1    datetime64[ns]
date2    datetime64[ns]
dtype: object


,date1,date2
0,2021-11-02,2021-11-02
1,2021-11-10,2021-11-10
2,2021-11-15,2021-11-15
3,2021-11-17,2021-11-17
4,2021-11-18,2021-11-18
